In [101]:
import pandas as pd
import yfinance as yf
import numpy as np 
from scipy.stats import gmean

In [88]:
portf = pd.read_csv("fin456_portfolio_holdings_t1.csv")

cash_row = portf[portf['TICKER'] == 'USD']
portf = portf[portf['TICKER'] != 'USD']

portf.tail()

,TICKER,QUANTITY
30,VICI,112.0
31,VIS,15.0
32,VOX,87.0
33,VPU,19.0
34,YUM,9.0


In [89]:
# Fetch data for the tickers in the portfolio
tickers = portf['TICKER'].tolist()
data = yf.download(tickers, start="2019-01-01", end="2024-01-01", interval="1mo")['Adj Close']

# Calculate monthly returns
returns = data.pct_change().dropna()


[*********************100%***********************]  35 of 35 completed


In [90]:
# Calculate total investment in each stock
current_prices = data.iloc[-1]  # Last row gives the latest prices
portf['Investment'] = portf['QUANTITY'] * current_prices.values

# Add the cash row back for total investment
total_investment = portf['Investment'].sum() + cash_row['QUANTITY'].iloc[0]

# Calculate weights
portf['Weight'] = portf['Investment'] / total_investment

print(portf[['TICKER', 'Weight']])

# Add a weight of cash (cash has zero returns)
cash_weight = cash_row['QUANTITY'].iloc[0] / total_investment
print(f"Cash Weight: {cash_weight:.2%}")

   TICKER    Weight
0    ADBE  0.025900
1    ATKR  0.015166
2     BBY  0.020808
3     BLK  0.020617
4     BRC  0.010075
5    CPAY  0.024537
6     CVI  0.007496
7     CVX  0.007525
8     FDX  0.019374
9   GOOGL  0.037505
10    JBL  0.017662
11    JPM  0.021769
12   LDOS  0.000929
13    LMT  0.019294
14    LOW  0.009481
15   LRCX  0.003930
16    MCK  0.016022
17    MET  0.024500
18    MOH  0.018822
19    MPC  0.006348
20    OVV  0.007051
21   SCHW  0.010595
22    TEL  0.021679
23    VAW  0.021093
24    VCR  0.041951
25    VDC  0.054858
26    VDE  0.007885
27    VFH  0.050276
28    VGT  0.150241
29    VHT  0.096543
30   VICI  0.029333
31    VIS  0.028324
32    VOX  0.087956
33    VPU  0.021870
34    YUM  0.010059
Cash Weight: 3.25%


In [92]:
# Calculate weighted monthly returns for the portfolio
weighted_returns = (returns * portf.set_index('TICKER')['Weight']).sum(axis=1)

# Adjust weighted returns to include cash (cash return is zero)
weighted_returns = weighted_returns * (1 - cash_weight)




In [97]:
weighted_returns.head()

Date
2019-02-01 00:00:00+00:00    0.033828
2019-03-01 00:00:00+00:00    0.007238
2019-04-01 00:00:00+00:00    0.052353
2019-05-01 00:00:00+00:00   -0.062471
2019-06-01 00:00:00+00:00    0.066234
dtype: float64

In [96]:
type(weighted_returns)

pandas.core.series.Series

In [99]:
weighted_returns.tolist()

[0.0338284448966324,
 0.0072379838818098004,
 0.0523530704342907,
 -0.062471313409919044,
 0.06623384855374803,
 0.021201032671907098,
 -0.02346902278805077,
 0.017272688664499047,
 0.02559412680174419,
 0.04773478104168551,
 0.021529171164478483,
 0.003847079710475537,
 -0.07948628939132078,
 -0.1455739458041799,
 0.1465312366337217,
 0.056200960728432066,
 0.01806169827390214,
 0.055713256169008664,
 0.05905191666206948,
 -0.03489950292570241,
 -0.007174074245142173,
 0.13143572384771166,
 0.0331046288160363,
 -0.0027378136930303124,
 0.05604877622385923,
 0.040914340423981045,
 0.05196642017956226,
 0.009919015385461643,
 0.011726107087764911,
 0.01769600414686912,
 0.030029477036874183,
 -0.045775802993864294,
 0.06071133270113996,
 -0.02243273280273715,
 0.04347832096049738,
 -0.03876298266941546,
 -0.022129617017320885,
 0.025609745984081975,
 -0.07262966515655382,
 0.01746708271187878,
 -0.08048223062633152,
 0.0849796006688954,
 -0.032492498489046354,
 -0.09482243789547905,
 0.

In [100]:
weighted_returns.to_numpy()

array([ 0.03382844,  0.00723798,  0.05235307, -0.06247131,  0.06623385,
        0.02120103, -0.02346902,  0.01727269,  0.02559413,  0.04773478,
        0.02152917,  0.00384708, -0.07948629, -0.14557395,  0.14653124,
        0.05620096,  0.0180617 ,  0.05571326,  0.05905192, -0.0348995 ,
       -0.00717407,  0.13143572,  0.03310463, -0.00273781,  0.05604878,
        0.04091434,  0.05196642,  0.00991902,  0.01172611,  0.017696  ,
        0.03002948, -0.0457758 ,  0.06071133, -0.02243273,  0.04347832,
       -0.03876298, -0.02212962,  0.02560975, -0.07262967,  0.01746708,
       -0.08048223,  0.0849796 , -0.0324925 , -0.09482244,  0.09308774,
        0.06007223, -0.05291612,  0.06928823, -0.02286254,  0.01943455,
        0.01000531, -0.00104394,  0.07230482,  0.04343903, -0.01754437,
       -0.03709547, -0.02579276,  0.07798176,  0.05105693])

In [106]:
weighted_returns.size

59

In [114]:
(1+weighted_returns).prod()**(1/59) - 1 # geometric monthly mean

np.float64(0.012461203764613016)

In [119]:
weighted_returns.mean() * 12 # arithmetic annual mean

np.float64(0.1670955962640911)

In [120]:
(1+weighted_returns).prod()**(12/weighted_returns.size) - 1 # geometric annual mean - this is the correct one

np.float64(0.16022090686374768)

In [116]:
12 * ((1+weighted_returns).prod()**(1/59) - 1) # incorrect - doesn't account for compounding

np.float64(0.1495344451753562)

In [117]:
((1+weighted_returns).prod()**(1/59) - 1)**12 # incorrect - not operating on it the proper way

np.float64(1.4019094766852057e-23)

In [111]:
(1+weighted_returns).prod()**(weighted_returns.size)

np.float64(5.274455600829477e+18)

In [103]:
gmean(weighted_returns, axis=None)

/home/allen/Desktop/portfolio_analyzer/portfolio_analyzer/.venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:210: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)


np.float64(nan)

In [94]:
# Expected annual return
expected_return = weighted_returns.mean() * 12
expected_return

np.float64(0.1670955962640911)

In [ ]:

print(f"Expected Annual Return: {expected_return:.2%}")

# Portfolio variance and standard deviation (risk)
portfolio_variance = np.dot(portf.set_index('TICKER')['Weight'].T, 
                            np.dot(returns.cov() * 12, 
                                   portf.set_index('TICKER')['Weight']))
portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
portfolio_std_dev = np.sqrt(portfolio_variance)
print(f"Portfolio Standard Deviation (Annualized): {portfolio_std_dev:.2%}")

In [62]:
# Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start="2019-01-01", end="2024-01-01", interval="1mo")['Adj Close']

# Calculate market monthly returns
market_returns = market_data.pct_change().dropna()


[*********************100%***********************]  1 of 1 completed


In [63]:
# Align both series and drop NaNs
aligned_portfolio_returns, aligned_market_returns = weighted_returns.align(market_returns, join='inner')

# Ensure no missing values remain after alignment
aligned_portfolio_returns = aligned_portfolio_returns.dropna()
aligned_market_returns = aligned_market_returns.dropna()

print(f"After cleaning: {len(aligned_portfolio_returns)} vs {len(aligned_market_returns)}")


After cleaning: 59 vs 59


In [64]:
# Ensure aligned_market_returns is a Series
if isinstance(aligned_market_returns, pd.DataFrame):
    aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series


In [65]:
cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns)

# Calculate beta
beta = cov_matrix[0, 1] / cov_matrix[1, 1]
print(f"Portfolio Beta: {beta:.2f}")


Portfolio Beta: 0.89


In [66]:
# We need risk free frate for calculating the sharpe ratio, so we're just gonna use yfinance and fetch TNX. If you want perfectly accurate sharpe calculation you can use U.S. treasury data from the internet for that. 10-yr bonds.
# Fetch 10-Year Treasury Yield (^TNX)
risk_free_data = yf.download('^TNX', start="2019-01-01", end="2024-01-01", interval="1mo")

# Check the structure of risk_free_data
print(risk_free_data.tail())  # Verify the last few rows of data

# Ensure you get the last valid yield as a scalar
risk_free_rate = risk_free_data['Adj Close'].dropna().iloc[-1] / 100  # Ensure single value and convert
risk_free_rate


[*********************100%***********************]  1 of 1 completed

Price                     Adj Close  Close   High    Low   Open Volume
Ticker                         ^TNX   ^TNX   ^TNX   ^TNX   ^TNX   ^TNX
Date                                                                  
2023-08-01 00:00:00+00:00     4.093  4.093  4.362  3.957  4.001      0
2023-09-01 00:00:00+00:00     4.573  4.573  4.688  4.060  4.097      0
2023-10-01 00:00:00+00:00     4.875  4.875  4.997  4.532  4.631      0
2023-11-01 00:00:00+00:00     4.352  4.352  4.903  4.253  4.893      0
2023-12-01 00:00:00+00:00     3.866  3.866  4.348  3.785  4.320      0


Ticker
^TNX    0.03866
Name: 2023-12-01 00:00:00+00:00, dtype: float64

In [73]:
# Sharpe Ratio
sharpe_ratio = (expected_return - risk_free_rate)/portfolio_std_dev
sharpe_ratio

# Interpretation
# Sharpe Ratio > 1: Good (your portfolio provides solid risk-adjusted returns).
# Sharpe Ratio > 2: Very good.
# Sharpe Ratio > 3: Excellent.
# Sharpe Ratio < 1: Risk-adjusted returns are relatively low; may need to re-evaluate risk.

Ticker
^TNX    0.638868
Name: 2023-12-01 00:00:00+00:00, dtype: float64

In [78]:
import numpy as np
import pandas as pd

In [86]:
a_list = pd.Series([0,2,5,6,7], index=[5,5,6,5,5])
a_list


5    0
5    2
6    5
5    6
5    7
dtype: int64

In [85]:
a_list.mean()

np.float64(4.0)